In [26]:
# For loading Reddit credentials
import os
from dotenv import load_dotenv

import pandas as pd
import numpy as np
import datetime as dt
#from pprint import pprint
#from itertools import chain

# reddit crawler
import praw

#sentiment analysis
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

#visualizations
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from wordcloud import WordCloud

In [27]:
load_dotenv()

True

In [28]:
reddit_client_id = os.getenv("REDDIT_CLIENT_ID")

reddit_client_secret = os.getenv("REDDIT_CLIENT_SECRET")

reddit_user_name = os.getenv("REDDIT_USER_NAME")

In [29]:
type(reddit_client_id)

str

In [30]:
nltk.download('vader_lexicon')
nltk.download('punkt')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\abhis\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\abhis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [31]:
# Setting up reddit client

r = praw.Reddit(user_agent = reddit_user_name,
               client_id = reddit_client_id,
                client_secret = reddit_client_secret
               )

In [35]:
# save the search term in a variable
search_term = input("Please provide the search term: ")
#As long as the search term is valid not empty...
if search_term != '':
    print(f'Searching for and analyzing {search_term}')
    
    # Empty list to create a Dataframe of results
    posts = []
    
    # Extract 10 new subreddits related to the search term
    subreddit = r.subreddit(search_term)
    news = [*subreddit.hot(limit = 10)] # Can also use new() or hot()
    
    #populating the dataframe with all the subreddit features 
    for post in news:
        posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, dt.datetime.fromtimestamp(post.created)])
    
post_df = pd.DataFrame(posts, columns = ['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'timestamp'])  
print(post_df)

Please provide the search term:  TSLA


Searching for and analyzing TSLA
                                               title  score      id subreddit  \
0                          Tesla moving up today! 🙌🏾     12  s0ub7z      TSLA   
1                           Robot trader picks Tesla     10  s0kdiw      TSLA   
2  Everyone knows that Elon is a key man at Tesla...     17  rzyel8      TSLA   
3                  Austin, we don't have a problem..      9  rzh6b6      TSLA   
4                                             Austin     14  rxp64c      TSLA   
5                               Tesla vs competition     13  rxdqmj      TSLA   
6                            Will Tesla split again?      0  rxjzld      TSLA   
7                           Nikola lawsuit dismissed      8  rwpsry      TSLA   
8                                      Tesla upgrade     10  rwl25r      TSLA   
9             Where/How do you buy your TSLA shares?      5  rwm9kq      TSLA   

                                                 url  num_comments  \
0  ht

In [36]:
sia = SentimentIntensityAnalyzer()

result = [*post_df['title'].apply(sia.polarity_scores)]
print(result[:3])

[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, {'neg': 0.0, 'neu': 0.758, 'pos': 0.242, 'compound': 0.7906}]


In [37]:
sentiment_df = pd.DataFrame.from_records(result)

reddit_news = pd.concat([post_df, sentiment_df], axis = 1, join='inner')

reddit_news.head()

,title,score,id,subreddit,url,num_comments,body,timestamp,neg,neu,pos,compound
0,Tesla moving up today! 🙌🏾,12,s0ub7z,TSLA,https://www.reddit.com/r/TSLA/comments/s0ub7z/...,2,Tesla is making moves upward. Take a position ...,2022-01-10 13:55:55,0.0,1.000,0.000,0.0000
1,Robot trader picks Tesla,10,s0kdiw,TSLA,https://www.reddit.com/r/TSLA/comments/s0kdiw/...,3,\n\n# Tesla and Nvidia Are Picked for Big Gai...,2022-01-10 06:46:17,0.0,1.000,0.000,0.0000
2,Everyone knows that Elon is a key man at Tesla...,17,rzyel8,TSLA,https://www.reddit.com/r/TSLA/comments/rzyel8/...,14,Drop names and rationale below,2022-01-09 11:48:03,0.0,0.758,0.242,0.7906
3,"Austin, we don't have a problem..",9,rzh6b6,TSLA,https://www.reddit.com/r/TSLA/comments/rzh6b6/...,8,Amazing news that mfg will be starting this we...,2022-01-08 19:32:39,0.0,1.000,0.000,0.0000
4,Austin,14,rxp64c,TSLA,https://www.reddit.com/r/TSLA/comments/rxp64c/...,5,"Buy with both hands, Austin firing up",2022-01-06 14:15:04,0.0,1.000,0.000,0.0000


In [39]:
threshold = 0.2

conditions = [(reddit_news['compound'] <= -0.2),
              (reddit_news['compound'] > -0.2) & (reddit_news['compound'] < threshold),
              (reddit_news['compound'] >= threshold)
             ]

values = ['neg', 'neu', 'pos']

reddit_news['label'] = np.select(conditions, values)

reddit_news

,title,score,id,subreddit,url,num_comments,body,timestamp,neg,neu,pos,compound,label
0,Tesla moving up today! 🙌🏾,12,s0ub7z,TSLA,https://www.reddit.com/r/TSLA/comments/s0ub7z/...,2,Tesla is making moves upward. Take a position ...,2022-01-10 13:55:55,0.000,1.000,0.000,0.0000,neu
1,Robot trader picks Tesla,10,s0kdiw,TSLA,https://www.reddit.com/r/TSLA/comments/s0kdiw/...,3,\n\n# Tesla and Nvidia Are Picked for Big Gai...,2022-01-10 06:46:17,0.000,1.000,0.000,0.0000,neu
2,Everyone knows that Elon is a key man at Tesla...,17,rzyel8,TSLA,https://www.reddit.com/r/TSLA/comments/rzyel8/...,14,Drop names and rationale below,2022-01-09 11:48:03,0.000,0.758,0.242,0.7906,pos
3,"Austin, we don't have a problem..",9,rzh6b6,TSLA,https://www.reddit.com/r/TSLA/comments/rzh6b6/...,8,Amazing news that mfg will be starting this we...,2022-01-08 19:32:39,0.000,1.000,0.000,0.0000,neu
4,Austin,14,rxp64c,TSLA,https://www.reddit.com/r/TSLA/comments/rxp64c/...,5,"Buy with both hands, Austin firing up",2022-01-06 14:15:04,0.000,1.000,0.000,0.0000,neu
5,Tesla vs competition,13,rxdqmj,TSLA,https://www.reddit.com/r/TSLA/comments/rxdqmj/...,5,"Porsche America sold 70,025 vehicles last year...",2022-01-06 05:43:53,0.000,1.000,0.000,0.0000,neu
6,Will Tesla split again?,0,rxjzld,TSLA,https://www.reddit.com/r/TSLA/comments/rxjzld/...,6,"I started investing a couple of years ago, but...",2022-01-06 10:31:24,0.000,1.000,0.000,0.0000,neu
7,Nikola lawsuit dismissed,8,rwpsry,TSLA,https://www.reddit.com/r/TSLA/comments/rwpsry/...,2,Both parties have dismissed the old $2b lawsui...,2022-01-05 08:45:38,0.487,0.513,0.000,-0.2263,neg
8,Tesla upgrade,10,rwl25r,TSLA,https://www.reddit.com/r/TSLA/comments/rwl25r/...,3,"Tesla stock price target raised to $1,300 fro...",2022-01-05 04:44:56,0.000,1.000,0.000,0.0000,neu
9,Where/How do you buy your TSLA shares?,5,rwm9kq,TSLA,https://www.reddit.com/r/TSLA/comments/rwm9kq/...,11,one of the online trade sites? thru your inves...,2022-01-05 05:53:48,0.000,0.732,0.268,0.2960,pos
